# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao construir a **pontuação de crédito** de um cliente em potencial. A **pontuação de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

[Neste caderno, você recebe dicas, instruções breves e sugestões de raciocínio. Não os ignore, pois eles são projetados para te equipar com a estrutura do projeto e o ajudarão a analisar o que você está fazendo em um nível mais profundo. Antes de enviar seu projeto, certifique-se de remover todas as dicas e descrições fornecidas a você. Em vez disso, faça com que este relatório pareça que você está enviando para seus colegas de equipe para demonstrar suas descobertas - eles não devem saber que você teve qualquer ajuda externa nossa! Para ajudá-lo, colocamos as dicas que você deve remover entre colchetes.]

[Antes de mergulhar na análise dos seus dados, explique os propósitos do projeto e as hipóteses que você testará.]

## Abra o arquivo de dados e veja a informação geral.

[Comece importando as bibliotecas e carregando os dados. Você pode perceber que precisa de bibliotecas adicionais à medida que avança, o que é completamente normal - apenas certifique-se de atualizar esta seção quando o fizer.]

In [5]:
# Carregando todas as bibliotecas

import pandas as pd

# Carregue os dados

try:
    df = pd.read_csv('/datasets/Projeto2/credit_scoring_eng.csv')
except:
    df = pd.read_csv('credit_scoring_eng.csv')
    

## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

[Agora vamos explorar nossos dados. Você vai querer ver quantas colunas e linhas ele tem, veja algumas linhas para verificar possíveis problemas com os dados.]

In [6]:
# usei o metodo shape para verificar quantas colunas e linhas existem
df.shape

(21525, 12)

In [7]:
# vamos imprimir as primeiras N linhas
#usei o metodo head() para imprimir as primeiras 5 linhas da tabela

df.head(5)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


[Descreva o que você vê e observa em sua amostra de dados exibida. Há algum problema que possa precisar de mais investigação e alterações?]

# Obter informações sobre dados
Há problemas na coluna days_employed e total_income existem menos dados do que o total ,há dados faltantes e precisamos entender o porque isso aconteceu.

[Existem valores ausentes em todas as colunas ou apenas em algumas? Descreva brevemente o que você vê em 1-2 frases.]

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Há duas colunas com valores ausentes,elas são days_employed e total_income

In [125]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df.loc[(df['days_employed'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


[Os valores ausentes parecem simétricos? Podemos ter certeza desta suposição? Explique brevemente seus pensamentos nesta seção. Você provavelmente pode querer realizar investigações adicionais e contar os valores ausentes em todas as linhas com valores ausentes para confirmar que as amostras ausentes são do mesmo tamanho.]

In [126]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.
df.loc[(df['days_employed'].isnull()) & (df['total_income'].isnull())]
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Os valores ausentes das colunas days_employed e total_income são simétricos, pois através da função feita vemos que nas linhas que contém valores ausentes/nulos na coluna days_employed ,também são ausentes/nulos na coluna total_income

**Conclusão intermediária**

[O número de linhas na tabela filtrada corresponde ao número de valores ausentes? Que conclusão podemos tirar disso?]

[Calcule a porcentagem dos valores ausentes em comparação com todo o conjunto de dados. É um pedaço de dados consideravelmente grande? Nesse caso, você pode querer preencher os valores ausentes. Para fazer isso, primeiro devemos considerar se os dados ausentes podem ser devidos à característica específica do cliente, como tipo de emprego ou outra coisa. Você precisará decidir qual característica você acha que pode ser o motivo. Em segundo lugar, devemos verificar se há alguma dependência de valores ausentes em relação ao valor de outros indicadores com as colunas com características identificadas específicas do cliente.]

[Explique seus próximos passos e como eles se correlacionam com as conclusões que você tirou até agora.]

In [127]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes

df_nulos = df.loc[(df['days_employed'].isnull()) & (df['total_income'].isnull())]
df_nulos

# Percebemos que há duas colunas com valores ausentes com dependência uma da outra a coluna total_income e days_employed!

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [128]:
total_nulos = df_nulos.shape[0]

In [129]:
dftotal = df.shape[0]

In [130]:
# A porcentagem de dados ausentes não é uma porcentagem grande, com isso encontramos os valores ausentes e os preenchemos !

percentual = (total_nulos / dftotal) * 100
percentual


10.099883855981417

In [131]:
# Verificar a distribuição

df_nulos.groupby('income_type')['dob_years'].count()

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: dob_years, dtype: int64

In [132]:
# Fiz um agrupamento para identificar os valores e ver se há valores ausentes!

df_nulos.groupby('income_type')['total_income'].count()

income_type
business         0
civil servant    0
employee         0
entrepreneur     0
retiree          0
Name: total_income, dtype: int64

**Possíveis motivos para valores ausentes nos dados**

[Vamos começar a verificar se os valores ausentes são aleatórios.]

In [133]:
# Verificar a distribuição em todo o conjunto de dados

df['total_income'].value_counts()

total_income
17312.717    2
42413.096    2
31791.384    2
21005.772    1
18591.443    1
            ..
27020.895    1
23686.835    1
9606.294     1
28156.762    1
13127.587    1
Name: count, Length: 19348, dtype: int64

**Conclusão intermediária**

[A distribuição no conjunto de dados original é semelhante à distribuição da tabela filtrada? O que isso significa para nós?]

[Se você acha que ainda não podemos tirar conclusões, vamos investigar melhor nosso conjunto de dados. Vamos pensar em outros motivos que podem levar à ausência de dados e verificar se podemos encontrar algum padrão que possa nos levar a pensar que os valores ausentes não são aleatórios. Como este é o seu trabalho, esta seção é opcional.]

In [1]:
# Verifique outros motivos e padrões que possam levar a valores ausentes
#A conclusão é que a pessoa não trabalhou e nao inseriu os dados por que ela nao quis 

**Conclusão intermediária**

[Podemos finalmente confirmar que os valores ausentes são acidentais? Verifique se há qualquer outra coisa que você acha que pode ser importante aqui.]

In [2]:
# Verificando outros padrões - explique quais? 
# Os valores ausentes 

**Conclusões**

[Você encontrou algum padrão? Como você chegou a essa conclusão?]

[Explique como você abordará os valores ausentes. Considere as categorias em que os valores estão ausentes.]

[Planeje brevemente suas próximas etapas para transformar dados. Você provavelmente precisará abordar diferentes tipos de problemas: duplicados, registros diferentes, artefatos incorretos e valores ausentes.]

O padrão é que nas mesmas linhas que há valores ausentes em days_employed também tem em total_income. 

## Transformação de dados

[Vamos examinar cada coluna para ver quais problemas podemos ter nelas.]

[Comece removendo duplicados e corrigindo informações educacionais, se necessário.]

In [136]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [137]:
# Corrija os registros, se necessário

df['education'] = df['education'].str.lower()

In [138]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

[Verifique os dados na coluna `children`]

In [139]:
# Vamos ver a distribuição de valores na coluna `children`= numero de crianças na familia

df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

[Há alguma coisa estranha na coluna? Se sim, qual é a porcentagem de dados problemáticos? Como eles podem ter ocorrido? Tome uma decisão sobre o que você fará com esses dados e explique seu raciocínio.]

In [140]:
# [corrija os dados com base na sua decisão]

df['children'] = df['children'].abs()

In [141]:
# Verificar a coluna `children` novamente para ter certeza de que está tudo corrigido

df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

[Verifique os dados na coluna `days_employed`. Em primeiro lugar, pense sobre que tipo de problemas pode haver, o que você pode querer verificar e como você fará isso.]

In [142]:
# Encontre dados problemáticos em `days_employed`, se existirem, e calcule a porcentagem = experiencia de trabalho em dias

df['days_employed'].value_counts()

# há valores negativos na experiência em dias de trabalho

days_employed
-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: count, Length: 19351, dtype: int64

[Se a quantidade de dados problemáticos for alta, pode ser devido a alguns problemas técnicos. Provavelmente, podemos querer propor a razão mais óbvia pela qual isso poderia ter acontecido e quais poderiam ter sido os dados corretos, pois não podemos descartar essas linhas problemáticas.]

In [143]:
# Aborde os valores problemáticos, se existirem

df['days_employed'] = df['days_employed'].abs()

In [144]:
# Verifique o resultado - certifique-se de que está corrigido

df['days_employed'].value_counts()

days_employed
8437.673028      1
3507.818775      1
354500.415854    1
769.717438       1
3963.590317      1
                ..
1099.957609      1
209.984794       1
398099.392433    1
1271.038880      1
1984.507589      1
Name: count, Length: 19351, dtype: int64

[Vamos agora olhar para a idade do cliente e se há algum problema aí. Novamente, pense em quais dados podem ser estranhos nesta coluna, ou seja, quais não podem ser a idade de alguém.]

In [145]:
# Verifique o `dob_years` para valores suspeitos e conte a porcentagem

df['dob_years'].value_counts()

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: count, dtype: int64

[Decida o que você fará com os valores problemáticos e explique por quê.]

In [146]:
# Resolva os problemas na coluna `dob_years`, se existirem
#tem 101 pessoas que tem 0 anos 

In [147]:
# Verifique o resultado - certifique-se de que está corrigido

[Agora vamos verificar a coluna `family_status`. Veja que tipo de valores existem e quais problemas você pode precisar resolver.]

In [148]:
# Vamos ver os valores da coluna

df['family_status'].value_counts()

#Não há problemas na coluna ,todos estão preenchidos ,por isso não há valores ausentes


family_status
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: count, dtype: int64

In [149]:
# Aborde os valores problemáticos em `family_status`, se eles existirem

#Não há valores problemáticos

In [150]:
# Verifique o resultado - certifique-se de que está corrigido

df['family_status'].isna().sum()

0

[Agora vamos verificar a coluna `gender`. Veja que tipo de valores existem e quais problemas você pode precisar resolver]

In [151]:
# Vamos ver os valores na coluna

df['gender'].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

In [152]:
# Aborde os valores problemáticos, se existirem
# excluir o valor xna 

df.drop(df[df['gender'] == 'XNA'].index, inplace=True)

In [153]:
df['gender'].value_counts()

gender
F    14236
M     7288
Name: count, dtype: int64

[Agora vamos verificar a coluna `income_type`. Veja que tipo de valores existem e quais problemas você pode precisar resolver]

In [154]:
# Vamos ver os valores na coluna

df['income_type'].value_counts()


income_type
employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

In [155]:
# Aborde os valores problemáticos, se existirem
#Não há valores problemáticos

In [156]:
# Verifique o resultado - certifique-se de que está corrigido

df['income_type'].isna().sum()

0

[Agora vamos ver se temos duplicados em nossos dados. Se o fizermos, você precisará decidir o que fará com eles e explicar o porquê.]

In [157]:
# Verificar duplicados

df.duplicated().sum()

71

In [158]:
# Corrija os duplicados, se existirem

df = df.drop_duplicates()

In [159]:
df.duplicated().sum()

0

In [160]:
#Verifique o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele
df.shape

(21453, 12)

[Descreva seu novo conjunto de dados: diga brevemente o que mudou e qual é a porcentagem das alterações, se houver.]


# Trabalhando com valores ausentes

[Para acelerar o trabalho com alguns dados, você pode querer trabalhar com dicionários para alguns valores, onde os IDs são fornecidos. Explique por quê e com quais dicionários você trabalhará.]

In [161]:
# Encontre os dicionários

dic_education = df.loc[:, ['education_id', 'education']]
dic_education = dic_education.drop_duplicates().reset_index(drop=True)
dic_education

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [162]:
dic_family = df.loc[:, ['family_status_id', 'family_status']]
dic_family = dic_family.drop_duplicates().reset_index(drop=True)
dic_family

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Restaurar valores ausentes em `total_income`

[Informe brevemente quais colunas têm valores ausentes que você precisa resolver. Explique como você irá corrigi-los.]


[Comece abordando os valores ausentes da receita total. Criar e categoria de idade para os clientes. Crie uma nova coluna com a categoria de idade. Essa estratégia pode ajudar no cálculo de valores para a renda total..]


In [163]:
# Vamos escrever uma função que calcule a categoria de idade

def age_category(age):
    if age < 0 or pd.isna(age):
        return 'NA'
    elif age < 10:
        return '0-9'
    elif age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    elif age < 70:
        return '60-69'
    else:
        return '70+'

In [164]:
# Teste se a função funciona
age_category(df['dob_years'].iloc[4])

'50-59'

In [165]:
# Criar coluna nova com base na função

df['age_group'] = df['dob_years'].apply(age_category)

In [166]:
# Verificar como ficaram os valores na nova coluna
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59


[Pense nos fatores dos quais a renda geralmente depende. Eventualmente, você desejará descobrir se deve usar valores médios ou medianos para substituir valores ausentes. Para tomar essa decisão, você provavelmente vai querer olhar para a distribuição dos fatores que você identificou como tendo impacto na renda de alguém.]

[Crie uma tabela que tenha apenas dados sem valores ausentes. Esses dados serão usados para restaurar os valores ausentes.]

In [167]:
# Crie uma tabela sem valores ausentes e imprima algumas de suas linhas para garantir que ela fique boa

df_sem_ausentes = df.loc[~df['days_employed'].isnull()]
df_sem_ausentes.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

In [168]:
# Veja os valores médios de renda com base nos seus fatores identificados

df_sem_ausentes.pivot_table(index=['income_type', 'gender'],
               columns= 'family_status',
               values='total_income',
               aggfunc='mean')

family_status                       civil partnership      divorced  \
income_type                 gender                                    
business                    F            29111.850129  32019.066557   
                            M            37156.671338  40092.561121   
civil servant               F            24410.351416  27619.634643   
                            M            33297.141135  37065.399667   
employee                    F            24143.062155  25278.054901   
                            M            28419.481650  27480.521574   
entrepreneur                F            79866.103000           NaN   
paternity / maternity leave F                     NaN           NaN   
retiree                     F            21537.383254  21637.408467   
                            M            25483.949703  28281.890286   
student                     M                     NaN           NaN   
unemployed                  F            32435.602000           NaN   
                            M                     NaN           NaN   

family_status                            married     unmarried  \
income_type                 gender                               
business                    F       29356.120319  29439.770244   
                            M       37934.581596  34651.520993   
civil servant               F       24455.442277  26841.646341   
                            M       34869.631157  30666.499055   
employee                    F       23261.892596  24927.162596   
                            M       30005.322433  25988.626269   
entrepreneur                F                NaN           NaN   
paternity / maternity leave F        8612.661000           NaN   
retiree                     F       21291.122230  22237.275924   
                            M       23961.412577  23512.900973   
student                     M                NaN  15712.260000   
unemployed                  F                NaN           NaN   
                            M        9593.119000           NaN   

family_status                       widow / widower  
income_type                 gender                   
business                    F          28813.660088  
                            M          27748.843167  
civil servant               F          24966.055225  
                            M          42925.538000  
employee                    F          24121.057815  
                            M          24041.476182  
entrepreneur                F                   NaN  
paternity / maternity leave F                   NaN  
retiree                     F          21281.813855  
                            M          18557.575429  
student                     M                   NaN  
unemployed                  F                   NaN  
                            M                   NaN

In [169]:
# Veja os valores medianos de renda com base nos seus fatores identificados

df_pivot = df_sem_ausentes.pivot_table(index=['income_type', 'gender'],
                   columns= 'family_status',
                   values='total_income',
                   aggfunc='median')
df_pivot

family_status                       civil partnership    divorced     married  \
income_type                 gender                                              
business                    F              25255.2075  26954.1980  25888.9010   
                            M              31010.1745  30945.7725  31818.0400   
civil servant               F              22598.8475  22967.2795  21572.1390   
                            M              31623.4095  33318.1010  29112.2945   
employee                    F              21503.1045  22180.4930  20432.6110   
                            M              25546.6040  24653.8220  26643.5125   
entrepreneur                F              79866.1030         NaN         NaN   
paternity / maternity leave F                     NaN         NaN   8612.6610   
retiree                     F              18251.1520  19185.7415  18296.2000   
                            M              20877.6770  21416.5410  21312.8770   
student                     M                     NaN         NaN         NaN   
unemployed                  F              32435.6020         NaN         NaN   
                            M                     NaN         NaN   9593.1190   

family_status                        unmarried  widow / widower  
income_type                 gender                               
business                    F       25907.9795       23208.7975  
                            M       30341.8220       27597.9835  
civil servant               F       23078.5040       20621.2900  
                            M       29002.9830       42925.5380  
employee                    F       20863.8790       22216.0035  
                            M       23825.1330       23433.5740  
entrepreneur                F              NaN              NaN  
paternity / maternity leave F              NaN              NaN  
retiree                     F       18905.6725       18854.1830  
                            M       19434.4580       18065.6980  
student                     M       15712.2600              NaN  
unemployed                  F              NaN              NaN  
                            M              NaN              NaN

[Repita essas comparações para vários fatores. Certifique-se de considerar diferentes aspectos e explicar seu processo de pensamento.]



Usei como referência a idade e o tipo de emprego pois para mim são os dois fatores que mais podem influênciar a renda das pessoas!

[Tome uma decisão sobre quais características definem mais a renda e se você usará uma mediana ou uma média. Explique por que você tomou essa decisão]


In [170]:
#  Escreva uma função que usaremos para preencher os valores ausentes

# df.groupby(['income_type', 'dob_years', 'days_employed'])['total_income'].mean().reset_index()

def aux_fill_nan(family_status, income_type, gender):
      try:
        return df_pivot[family_status][income_type][gender]
      except:
        return 'erro'

In [171]:
# Verifique se funciona

aux_fill_nan('civil partnership', 'business','F')

25255.207500000004

In [172]:
# Aplique em todas as linhas
df['mediana_total'] = df.apply(lambda row: aux_fill_nan(row['family_status'], row['income_type'], row['gender']), axis=1)

In [173]:
# Verifique se temos algum erro
df['total_income'] = df['total_income'].fillna(df['mediana_total'])

In [174]:
df[df['total_income']=='erro']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,mediana_total
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,erro,buy residential real estate,50-59,erro


[Se você encontrou erros ao preparar os valores para dados ausentes, provavelmente significa que há algo especial nos dados da categoria. Pense um pouco - você pode querer corrigir algumas coisas manualmente, se houver dados suficientes para encontrar medianas/médias.]


[Quando você achar que terminou com `total_income`, verifique se o número total de valores nesta coluna corresponde ao número de valores em outras.]

In [175]:
# Substituir valores ausentes se houver algum erro 

df['total_income'] = df['total_income'].replace('erro', 31604.177265)

In [176]:
df['total_income'].dtype

dtype('float64')

In [177]:
df['total_income'].isna().sum()

0

In [178]:
# Verificar o número de entradas nas colunas


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21453 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_group         21453 non-null  object 
 13  mediana_total     21453 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 2.5+ MB


###  Restaurar valores em `days_employed`

[Pense nos parâmetros que podem ajudá-lo a restaurar os valores ausentes nesta coluna. Eventualmente, você desejará descobrir se deve usar valores médios ou medianos para substituir valores ausentes. Você provavelmente fará uma pesquisa semelhante à que fez ao restaurar dados em uma coluna anterior.]

In [179]:
# Distribuição de `days_employed` medianos com base nos seus parâmetros identificados

dados_days = df.pivot_table(index=['income_type','family_status'],
                                   columns='gender',
                                   values='days_employed',
                                   aggfunc='median')
dados_days

gender                                                     F              M
income_type                 family_status                                  
business                    civil partnership    1406.527738    1359.016964
                            divorced             1520.532918    1837.252685
                            married              1686.419677    1587.696641
                            unmarried            1305.150018    1069.370722
                            widow / widower      2757.584181    1549.012430
civil servant               civil partnership    2645.791195    2412.218805
                            divorced             2979.425067    2751.423468
                            married              2758.292954    2914.292293
                            unmarried            2165.130798    2103.355440
                            widow / widower      2830.224607    3046.540954
employee                    civil partnership    1726.948357    1289.359088
                            divorced             1629.346256    1319.492928
                            married              1809.885076    1530.385958
                            unmarried            1294.277533    1020.749261
                            widow / widower      2083.456926    1293.823205
entrepreneur                civil partnership     520.848083            NaN
paternity / maternity leave married              3296.759962            NaN
retiree                     civil partnership  367793.922188  363581.636450
                            divorced           366355.800736  369526.172009
                            married            365311.591487  361002.065406
                            unmarried          366479.537413  372850.498803
                            widow / widower    365928.378199  352557.868347
student                     unmarried                    NaN     578.751554
unemployed                  civil partnership  395302.838654            NaN
                            married                      NaN  337524.466835

In [180]:
# Distribuição de `days_employed` médios com base nos seus parâmetros identificados

df.pivot_table(index=['income_type', 'family_status'],
                                         columns='gender',
                                         values='days_employed',
                                         aggfunc='mean'
)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,mediana_total
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,20432.611
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,20432.611
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,26643.5125
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,26643.5125
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,18251.152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49,25255.2075
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,18296.2
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39,25546.604
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,26643.5125


[Decida o que você usará: médias ou medianas. Explique por quê.]

Usarei mediana ,não há interferência em valores 

In [181]:
# Vamos escrever uma função que calcule médias ou medianas (dependendo da sua decisão) com base no seu parâmetro identificado

def median_days(gender, income_type, family_status):
    try:
        return df[gender][income_type][family_status]
    except:
        return 'Erro'

In [182]:
# Verifique se a função funciona

median_days(gender='F', income_type='business', family_status='civil patnership')

'Erro'

In [183]:
# Aplicar função ao income_type

df['days_employed'] = df.apply(lambda row: median_days(gender=row['gender'],
                                                       income_type=row['income_type'],
                                                       family_status=row['family_status']),
                                               axis=1)

In [184]:
# tirando o filtro homens da coluna gender

df[(df['gender'] == 'F') & (df['days_employed'] != 'Erro')]['days_employed'].median()

nan

In [185]:
df[df['days_employed'] == 'Error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,mediana_total


In [186]:
# Verifique se a função funcionou

df['days_employed'].isnull().sum()

0

In [187]:
# Substituir valores ausentes

df['days_employed'].isna().sum()

0

[Quando você achar que terminou com `total_income`, verifique se o número total de valores nesta coluna corresponde ao número de valores em outras.]

In [188]:
# Verifique as entradas em todas as colunas - certifique-se de corrigir todos os valores ausentes

df.count()

children            21453
days_employed       21453
dob_years           21453
education           21453
education_id        21453
family_status       21453
family_status_id    21453
gender              21453
income_type         21453
debt                21453
total_income        21453
purpose             21453
age_group           21453
mediana_total       21453
dtype: int64

## Categorização de dados

[Para responder às perguntas e testar as hipóteses, você vai querer trabalhar com dados categorizados. Veja as perguntas que lhe foram colocadas e que você deve responder. Pense em quais dados precisarão ser categorizados para responder a essas perguntas. Abaixo, você encontrará um modelo por meio do qual poderá trabalhar à sua maneira ao categorizar os dados. O primeiro processamento passo a passo abrange os dados de texto; o segundo aborda os dados numéricos que precisam ser categorizados. Você pode usar ambas ou nenhuma das instruções sugeridas - você decide.]

[Independentemente de como você decidir abordar a categorização, certifique-se de fornecer uma explicação clara do por quê você tomou sua decisão. Lembre-se: este é o seu trabalho e você toma todas as decisões nele.]


In [189]:
# Imprima os valores dos dados selecionados para categorização
df.groupby('family_status')['total_income'].count()

family_status
civil partnership     4150
divorced              1195
married              12339
unmarried             2810
widow / widower        959
Name: total_income, dtype: int64

In [190]:
# pelo numero de linhas 
df.groupby('children')['total_income'].count()

children
0     14090
1      4855
2      2052
3       330
4        41
5         9
20       76
Name: total_income, dtype: int64

[Vamos verificar valores exclusivos]

In [191]:
# Verifique os valores exclusivos

df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [192]:
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

In [193]:
sorted(df['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

[Quais grupos principais você pode identificar com base nos valores exclusivos?]

[Com base nesses temas, provavelmente desejaremos categorizar nossos dados.]


In [194]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comunss

def category_df(purpose):
    if 'car' in purpose:
        return 'carro'
    elif 'real estate' in purpose or 'property' in purpose or 'housing' in purpose:
        return 'Imóvel'
    elif 'education' in purpose or 'university' in purpose:
        return 'Educação'
    elif 'wedding' in purpose:
        return 'Casamento'
    else:
        return 'Outros'

In [195]:
# Crie uma coluna com as categorias e conte os valores para elas

df['category_df'] = df['purpose'].apply(category_df)

In [196]:
df['category_df'].value_counts()

category_df
Imóvel       8906
carro        4306
Educação     3605
Casamento    2324
Outros       2312
Name: count, dtype: int64

[Se você decidir categorizar os dados numéricos, também precisará criar as categorias para isso.]

In [197]:
# Examinar todos os dados numéricos em sua coluna selecionada para categorização

df['purpose'].describe()

count                21453
unique                  38
top       wedding ceremony
freq                   791
Name: purpose, dtype: object

In [198]:
# Obter estatísticas resumidas para a coluna

df['purpose'].describe()

count                21453
unique                  38
top       wedding ceremony
freq                   791
Name: purpose, dtype: object

In [199]:
# Criar função para categorização em diferentes grupos numéricos com base em intervalos

def client_category(total_income):
    if total_income <= 20000:
        return 'Low Income'
    elif total_income <= 50000:
        return 'Median'
    else:
        return 'Higth'

In [200]:
client_category(17932.802)

'Low Income'

In [201]:
# Criar coluna com categorias
df['client_category'] = df.apply(lambda row: client_category(row['total_income']), axis=1)

In [202]:
df.groupby('client_category').agg({'debt' : 'mean'})

,debt
client_category,
Higth,0.069697
Low Income,0.082608
Median,0.081470


In [203]:
# Conte os valores de cada categoria para ver a distribuição

df['client_category'].value_counts()

client_category
Median        12434
Low Income     7699
Higth          1320
Name: count, dtype: int64

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [204]:
# Verifique os dados das crianças e do pagamento em dia
# Calcular a taxa de inadimplência com base no número de filhos
# Percebemos que quanto mais crianças tem na família 

df.groupby('children').agg({'debt':['mean' ,'count']})

debt       
              mean  count
children                 
0         0.075444  14090
1         0.091658   4855
2         0.094542   2052
3         0.081818    330
4         0.097561     41
5         0.000000      9
20        0.105263     76

**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]


Conclui que com o que foi construído há bastante mudança débitos quando se vê a quantidade de crianças que tem na família ,fazendo com que os gastos aumentem,pelo fato que quanto mais crianças mais gastos você tem com escola,alimentação , roupa e outros!

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [3]:
# Verifique os dados de status da família e do pagamento em dia

def criar_tabela(s):
    return df.groupby(s).agg({'debt':['mean', 'count']})

# Calcular a taxa padrão com base no status da família
# Percebemos um correlação que sendo solteira e parceira civil há dos pagamentos é maior comparado aos casados ,divorciados e viúva

In [206]:
 criar_tabela('family_status')

debt       
                       mean  count
family_status                     
civil partnership  0.093494   4150
divorced           0.071130   1195
married            0.075452  12339
unmarried          0.097509   2810
widow / widower    0.065693    959

**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]

O status de parceiro civil tem a média de um débito também elevada ,isso podde acontecer por conta dos gastos de uma vida há dois ,como contas de casa ,roupas ,alimentação ,gastos com o carro e é por esses e outros motivos que tem uma das médias de gastos maiores!

Observei que o débito financeiro da solteira é maior ,isso pode acontecer por conta de gastos com casa, festas ,alimentação e como a renda dela é única ,somente ela gasta !

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [207]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os

def taxa_cliente(s):
    return df.groupby(s).agg({'debt':['mean', 'count']})

# Percebemos que quando a finalidade do crédito é para algo relacionado ao carro a média do débito é maior 

In [208]:
taxa_cliente('purpose')

debt      
                                              mean count
purpose                                                 
building a property                       0.087237   619
building a real estate                    0.076923   624
buy commercial real estate                0.071104   661
buy real estate                           0.069355   620
buy residential real estate               0.067657   606
buying a second-hand car                  0.075314   478
buying my own car                         0.091089   505
buying property for renting out           0.079877   651
car                                       0.085020   494
car purchase                              0.091106   461
cars                                      0.092050   478
construction of own property              0.066142   635
education                                 0.071588   447
getting an education                      0.083710   442
getting higher education                  0.107981   426
going to university                       0.086694   496
having a wedding                          0.083333   768
housing                                   0.071207   646
housing renovation                        0.057661   607
housing transactions                      0.073620   652
profile education                         0.100917   436
property                                  0.066351   633
purchase of a car                         0.109890   455
purchase of my own house                  0.054839   620
purchase of the house                     0.074303   646
purchase of the house for my family       0.070533   638
real estate transactions                  0.081481   675
second-hand car purchase                  0.104938   486
supplementary education                   0.082609   460
to become educated                        0.095588   408
to buy a car                              0.093418   471
to get a supplementary education          0.114350   446
to have a wedding                         0.075817   765
to own a car                              0.100418   478
transactions with commercial real estate  0.080000   650
transactions with my real estate          0.079745   627
university education                      0.088496   452
wedding ceremony                          0.080910   791

Concluimos que o maior valor de gastos é usado para algo relacionado ao carro ,tanto para compra ou manutenção ,fazendo com que a média de pessoas que pegam dinheiro para algo relacionado a carro seja uma das maiores!

**Conclusão**

[Escreva suas conclusões com base em suas manipulações e observações.]


# Conclusão Geral 

[Liste suas conclusões nesta seção final. Certifique-se de incluir todas as suas conclusões importantes que o levaram à maneira como você processou e analisou os dados. Cubra os valores ausentes, duplicados e possíveis razões e soluções para artefatos problemáticos que você teve que resolver.]

[Liste suas conclusões sobre as perguntas feitas aqui também.]


Percebemos através da análise que a quantidade de crianças na família,se a pessoa é casada ou não, a idade da pessoa e o nível de educação afeta a renda do cliente ,com isso são bem interligados ,dependendo um do outro!

Os valores duplicados e ausentes pode ter acontecido por vários fatos, sendo por talvez não ter colocado 

In [8]:
df.to_csv('credit_scoring_eng_clean.csv', index=False)